In [2]:
import pandas as pd
import random
import itertools

In [3]:
# Function to simulate a prisoner's dilemma game
def play_prisoners_dilemma(contestants, iterations=5):
    # Initialize a DataFrame to store results
    results = pd.DataFrame(columns=['Contestant1', 'Contestant2', 'Round', 'C1_Decision', 'C2_Decision', 'C1_Payoff', 'C2_Payoff'])

    # Define payoff matrix
    payoff_matrix = {
        ('Cooperate', 'Cooperate'): (2, 2),
        ('Cooperate', 'Defect'): (0, 3),
        ('Defect', 'Cooperate'): (3, 0),
        ('Defect', 'Defect'): (1, 1)
    }

    # Initialize a dictionary to store total scores
    total_scores = {contestant: 0 for contestant in contestants.keys()}

    # Initialize a dictionary to store matchup scores
    matchup_scores = {c1: {c2: 0 for c2 in contestants.keys()} for c1 in contestants.keys()}

    # Iterate over all pairs of contestants
    for c1, c2 in itertools.combinations(contestants.keys(), 2):
        # Initialize history DataFrames for both contestants
        c1_history = pd.DataFrame(columns=['Round', 'Opponent_Decision', 'My_Decision'])
        c2_history = pd.DataFrame(columns=['Round', 'Opponent_Decision', 'My_Decision'])

        for round_number in range(1, iterations + 1):
            # Get decisions from both contestants
            c1_decision = contestants[c1](c1_history)
            c2_decision = contestants[c2](c2_history)

            # Record decisions in results DataFrame
            c1_payoff, c2_payoff = payoff_matrix[(c1_decision, c2_decision)]
            new_result = pd.DataFrame([{
                'Contestant1': c1,
                'Contestant2': c2,
                'Round': round_number,
                'C1_Decision': c1_decision,
                'C2_Decision': c2_decision,
                'C1_Payoff': c1_payoff,
                'C2_Payoff': c2_payoff
            }])
            results = pd.concat([results, new_result], ignore_index=True)

            # Update total scores
            total_scores[c1] += c1_payoff
            total_scores[c2] += c2_payoff

            # Update matchup scores
            matchup_scores[c1][c2] += c1_payoff
            matchup_scores[c2][c1] += c2_payoff

            # Update history DataFrames
            new_c1_history = pd.DataFrame([{'Round': round_number, 'Opponent_Decision': c2_decision, 'My_Decision': c1_decision}])
            new_c2_history = pd.DataFrame([{'Round': round_number, 'Opponent_Decision': c1_decision, 'My_Decision': c2_decision}])
            c1_history = pd.concat([c1_history, new_c1_history], ignore_index=True)
            c2_history = pd.concat([c2_history, new_c2_history], ignore_index=True)

    return results, total_scores, matchup_scores



In [17]:
# Contestant strategies
def pushover(history):
    return 'Cooperate'

def gaslighter(history):
    """
    Cooperates the first three turns, then defects thereafter.
    """
    if history.empty or len(history) < 3:
        return 'Cooperate'
    return 'Defect'

def north_korea(history):
    """
    Defects the first three times, then cooperates the next three times, and so on.
    """
    # Determine the current round
    current_round = len(history) + 1

    # Determine the phase of the strategy
    phase = (current_round - 1) // 5

    if phase % 2 == 0:
        return 'Defect'
    else:
        return 'Cooperate'

def Sam(history):
    if not history.empty:
        if history.Round.max() == 1 or history.Round.max() == 4:
            return "Cooperate"
    return "Defect"

def ryan(history):
    return 'Defect'

def Steve(history):
    if not history.empty:
        if list(history.sort_values(by="Round").Opponent_Decision)[:5] == ["Defect", "Cooperate", "Defect", "Defect", "Cooperate"]:
            return "Cooperate"
    return "Defect"

def wenng(history):
    """
    truly a horrible mindset but it is what it is in a cut throat world...
    """
    if not history.empty:
        if 'Defect' in history['Opponent_Decision'].values:
            return 'Defect'
        else:
            return 'Cooperate'
    return 'Cooperate'

def fat_finger(history):
    import random
    choice_list = ['Cooperate', 'Defect']
    wts = [1,19]
    choice = random.choices(choice_list, weights=wts)
    return(choice[0])

def Con_Air(history):
    return 'Defect'

def Saad(history):
    return 'Defect'

def punisher(history):
    current_round = history.shape[0] + 1
    if current_round == 1:
        return "Cooperate"
    elif current_round < 50:
        opp_decs = set(history.Opponent_Decision)
        if "Cooperate" in opp_decs and len(opp_decs) == 1:
            return "Cooperate"
        else:
            return "Defect" 
    else:
        return "Defect"
    
def jon_k(history):
    if not history.empty:
        if list(history.sort_values(by="Round").Opponent_Decision)[:5] == ["Defect", "Cooperate", "Defect", "Defect", "Cooperate"]:
            return "Cooperate"
    return "Defect"

def paffshank_redemption(df):
    #Overly complicated to make it interesting

    # First round
    if df.empty:
        return 'Cooperate'  

    current_round = df['Round'].iloc[-1] + 1
    rounds_left = 50 - current_round

    opponent_history = df['Opponent_Decision'].tolist()
    my_history = df['My_Decision'].tolist()

    # End game strat
    # Always defect in the last x rounds
    if rounds_left < 1:
        return 'Defect'

    # Always defect if opponent defected in first two rounds - no mercy
    if current_round > 2 and opponent_history[:2] == ['Defect', 'Defect']:
        return 'Defect'

    # Adaptive defection - add some spice
    # Take advantage of forgiving functions
    # Try first defection on round 10(increase?)
    if current_round == 10:
        return 'Defect'

    # If not punished for defection increase frequency of defection
    unpunished_defections = 0
    for i in range(len(my_history) - 1):
        if my_history[i] == 'Defect' and opponent_history[i + 1] == 'Cooperate':
            unpunished_defections += 1
    if unpunished_defections > 0:
        defection_frequency = max(10 - unpunished_defections, 2)  # Frequency ranges from 2 to 10
        if current_round % defection_frequency == 0:
            return 'Defect'

    # Attempt Pattern recognition
    # Opponent might be alternating
    if opponent_history[-5:] == ['Cooperate', 'Defect'] * 2 + ['Cooperate']:
        return 'Defect'  

    # Check if we're in a tit-for-tat feedback loop
    def in_alternating_pattern():
        if len(my_history) >= 4:
            if my_history[-4:] == ['Defect', 'Cooperate', 'Defect', 'Cooperate']:
                return True
        return False

    # Punish if opponent has ever defected after we attempted to reconcile
    def opponent_defected_after_cooperation():
        for i in range(len(my_history) - 1):
            if my_history[i] == 'Cooperate' and opponent_history[i + 1] == 'Defect':
                return True
        return False

    # Anti-bounce mechanism
    # Attempt to break the cycle by cooperating
    if current_round > 4 and in_alternating_pattern():
        # If opponent has ever defected after we cooperated, no forgiveness
        if opponent_defected_after_cooperation():
            return 'Defect'
        else:
            return 'Cooperate'  

    # Mid-game strat - make it more interesting
    # Punitive against frequent Defectors
    cooperation_ratio = opponent_history.count('Cooperate') / len(opponent_history)

    # Adjust cooperation threshold based on rounds left
    cooperation_threshold = 0.7 + (rounds_left / 50) * 0.1

    if cooperation_ratio < 0.3:
        return 'Defect'

    # Tit-for-tat as a fallback strat zzzzzz
    return opponent_history[-1]

def neo_mar(history):
    if not history.empty:
        if list(history.sort_values(by="Round").Opponent_Decision)[:5] == ["Defect", "Cooperate", "Defect", "Defect", "Cooperate"]:
            return "Cooperate"
    return "Defect"

def Julia(history):
    import numpy as np
    import sys
    np.set_printoptions(threshold=sys.maxsize)
    np.set_printoptions(linewidth=999999)


##    import matplotlib.pyplot as plt

    N_rounds = 5
    if history.empty == 1:
        my_fraction = np.random.random()
        their_fraction = np.random.random()
    else:
        MAX = max(history['Round'])
        MIN = np.clip(MAX - N_rounds, a_min=0, a_max=None)

##        print(history.loc[
##            (history['Round'] >= MIN) & (history['Round'] <= MAX),
##            :
##        ])

        my_fraction = sum(history.loc[
            (history['Round'] >= MIN) & (history['Round'] <= MAX),
            'My_Decision'
        ] == 'Cooperate')/(MAX - MIN)
        my_fraction += np.random.normal(0, 0.1)
        my_fraction = np.clip(my_fraction, a_min=0, a_max=1)

        their_fraction = sum(history.loc[
            (history['Round'] >= MIN) & (history['Round'] <= MAX),
            'Opponent_Decision'
        ] == 'Cooperate')/(MAX - MIN)

    if (my_fraction == 1) | (my_fraction == 0):
        my_fraction = np.random.random()

    c = complex(1.5*(my_fraction-0.5), 1.5*(their_fraction-0.5))

    w = 200
    h = 200
    re_min, re_max = -1.5, 1.5
    im_min, im_max = -1.5, 1.5
    real_range = np.arange(re_min, re_max, (re_max - re_min) / w)
    imag_range = np.arange(im_max, im_min, (im_min - im_max) / h)

    point_x = w*their_fraction + np.random.normal(0, 10)
    point_x = np.clip(int(round(point_x)), a_min=0, a_max=w)
    point_y = h*my_fraction + np.random.normal(0, 10)
    point_y = np.clip(int(round(point_y)), a_min=0, a_max=h)

    img = np.ones([w+1, h+1])*255
    for y, im in enumerate(imag_range):
        for x, re in enumerate(real_range):
            z = complex(re, im)
            n = 255
            while abs(z) < 10 and n >= 5:
                z = z*z + c
                n -= 5
            img[y, x] = n

    if img[point_y, point_x] == 0:
        result = "Cooperate"
    else:
        result = "Defect"

##    plt.imshow(img)
##    plt.scatter(point_x, point_y, c='r')
##    plt.title(result)
##    plt.show(block=False)
##    plt.pause(0.5)
##    plt.close()

    return result

def Jono(history):
    """
    Cooperates on the first move, then mimics the opponent's previous move.
    """
    if history.empty:
        return 'Cooperate'
    else:
        return history['Opponent_Decision'].iloc[-1]


# Define contestants
contestants = {
    'gaslighter': gaslighter,
    'pushover': pushover,
    'north korea': north_korea,
    'Sam': Sam,
    'ryan': ryan,
    'Steve': Steve,
    'wenng': wenng,
    'fat_finger': fat_finger,
    'Con Air': Con_Air,
    'Saad': Saad,
    'punisher': punisher,
    'jon k': jon_k,
    'paffshank_redemption': paffshank_redemption,
    'neo mar': neo_mar,
    'Julia Fractal': Julia,
    'Jono': Jono,
}

In [18]:
# Run the prisoner's dilemma simulation
results, total_scores, matchup_scores = play_prisoners_dilemma(contestants, iterations=20)


Matchup Results:
        Contestant1 Contestant2 Round C1_Decision C2_Decision C1_Payoff  \
0        gaslighter    pushover     1   Cooperate   Cooperate         2   
1        gaslighter    pushover     2   Cooperate   Cooperate         2   
2        gaslighter    pushover     3   Cooperate   Cooperate         2   
3        gaslighter    pushover     4      Defect   Cooperate         3   
4        gaslighter    pushover     5      Defect   Cooperate         3   
...             ...         ...   ...         ...         ...       ...   
2395  Julia Fractal        Jono    16      Defect   Cooperate         3   
2396  Julia Fractal        Jono    17      Defect      Defect         1   
2397  Julia Fractal        Jono    18      Defect      Defect         1   
2398  Julia Fractal        Jono    19   Cooperate      Defect         0   
2399  Julia Fractal        Jono    20   Cooperate   Cooperate         2   

     C2_Payoff  
0            2  
1            2  
2            2  
3            0

In [23]:
# Display results
print("Matchup Results:")
print(results)

# Display the score matrix with hyphens for self-intersections
matchup_scores_df = pd.DataFrame(matchup_scores).fillna(0).astype(int).astype(str)

# Substitute diagonal values with hyphens
for contestant in contestants.keys():
    matchup_scores_df.loc[contestant, contestant] = '-'
matchup_scores_df = matchup_scores_df.T

print("\nScore Matrix:")
print(matchup_scores_df)

# Rank the contestants by their total scores
total_scores_df = pd.DataFrame(list(total_scores.items()), columns=['Contestant', 'Total_Score'])
ranked_scores_df = total_scores_df.sort_values(by='Total_Score', ascending=False).reset_index(drop=True)
print("\nRanked Scores:")
print(ranked_scores_df)


Matchup Results:
        Contestant1 Contestant2 Round C1_Decision C2_Decision C1_Payoff  \
0        gaslighter    pushover     1   Cooperate   Cooperate         2   
1        gaslighter    pushover     2   Cooperate   Cooperate         2   
2        gaslighter    pushover     3   Cooperate   Cooperate         2   
3        gaslighter    pushover     4      Defect   Cooperate         3   
4        gaslighter    pushover     5      Defect   Cooperate         3   
...             ...         ...   ...         ...         ...       ...   
2395  Julia Fractal        Jono    16      Defect   Cooperate         3   
2396  Julia Fractal        Jono    17      Defect      Defect         1   
2397  Julia Fractal        Jono    18      Defect      Defect         1   
2398  Julia Fractal        Jono    19   Cooperate      Defect         0   
2399  Julia Fractal        Jono    20   Cooperate   Cooperate         2   

     C2_Payoff  
0            2  
1            2  
2            2  
3            0

In [26]:
print(matchup_scores_df.iloc[0].name)
matchup_scores_df.iloc[0]

gaslighter


gaslighter               -
pushover                57
north korea             37
Sam                     21
ryan                    17
Steve                   17
wenng                   25
fat_finger              17
Con Air                 17
Saad                    17
punisher                25
jon k                   17
paffshank_redemption    25
neo mar                 17
Julia Fractal           17
Jono                    25
Name: gaslighter, dtype: object